In [ ]:
#TEST Getting Information
#DATE 13 May 2025

#Version 1.3 - Funcional para avance

'''
# Esta version incluye: 
1. Procesamiento de la foto con EASYOCR
2. Creacion del Df
3. Obtencion de valores separados o juntos

Excluye:
4. Excluye visualizaciones
5. Excluye control de palabras similares




CONTIENE: 
A. Primera version de obtención de valores
'''

In [2]:
import os
import numpy as np
import easyocr
import pandas as pd #Para Dataframes y manipulacion de los valores
import re           #Manipulacion de caracteres

In [3]:
#image_path = '/Users/fer/Downloads/Proyecto Integrador (DEL)/All photos/Fotos_Etiquetas_1/1741038478979.jpg'  #Tiene REF, LOT, GTIN
image_path = '/Users/fer/Downloads/Proyecto Integrador (DEL)/All photos/Fotos_Etiquetas_1/1741038479261.jpg'   #Tiene REF, LOT

print("Esta la imagen?", os.path.exists(image_path)) ##Confirmar si la imagen existe


Esta la imagen? True


In [4]:
###################
# SECCION 1: Obtencion de Texto usando EasyOCR y creacion del DF
###################
#Crear lector de EasyOCR
reader = easyocr.Reader(['es', 'en'])  #Español e ingles por lo pronto

results = reader.readtext(image_path)  #Leer texto desde la imagen

results_df = pd.DataFrame(results)     #Crear dataframe para procesamiento
results_df.head()


,0,1,2
0,"[[1907, 849], [2519, 849], [2519, 948], [1907,...",Bottle Set small,0.766554
1,"[[1920, 989], [2032, 989], [2032, 1056], [1920...",REF,0.999810
2,"[[2055, 971], [2588, 971], [2588, 1080], [2055...",(240)07254733190,0.830424
3,"[[1917, 1113], [2019, 1113], [2019, 1174], [19...",LOT,0.640695
4,"[[2056, 1095], [2360, 1095], [2360, 1182], [20...",(10)81259101,0.703669


In [6]:
###### Funcion para extraer fechas en formato AAAA-MM-DD
def extraer_valores_por_tag(results_df):
    #Uso de Regex para el formato de fecha
    fecha_patron = r"\b\d{4}-\d{2}-\d{2}\b"
    fechas = []

    #Ciclo FOR sobre los textos extraidos
    for idx, texto in results_df[1].items():
        if pd.isna(texto):
            continue
        #Buscar fechas en el textoo
        fechas_encontradas = re.findall(fecha_patron, texto)
        # Agregar fechas encontradas
        fechas.extend(fechas_encontradas)

    # Eliminar duplicados y ordenar por fecha
    fechas = sorted(list(set(fechas)))

    # Determinar FAB y CAD
    fab_fecha = fechas[0] if len(fechas) > 0 else "NA"
    cad_fecha = fechas[1] if len(fechas) > 1 else "NA"

    return fab_fecha, cad_fecha

##### Funcion para buscar valores por etiquetas
def extraer_valores_por_tag(results_df, tags):
    valores = {tag: "NA" for tag in tags}

    for tag in tags:
        for idx, texto in results_df[1].items():
            if pd.isna(texto):
                continue

            #Se limpia el texto para comparacion
            clean_texto = texto.upper().replace("[", "").replace("]", "")

            #Caso 1: Si es exactamente igual (etiqueta sola), tomamos siguiente entrada
            if clean_texto.strip() == tag:
                if idx + 1 in results_df.index:
                    valores[tag] = results_df.loc[idx + 1, 1]
                break

            #Caso 2: Si la etiqueta viene combinada con el valor
            elif tag in clean_texto:
                match = re.search(rf"{tag}\s*[:\-]?\s*([A-Z0-9]+)", clean_texto)
                if match:
                    valores[tag] = match.group(1)
                    break
                partes = clean_texto.split()
                if len(partes) > 1 and tag in partes[0]:
                    valores[tag] = partes[1]
                    break

    return valores

In [10]:
###################
# SECCION 2: Obtencion de valores desde el DF
###################

tags_list = ["REF", "LOT"]
tags_valores = extraer_valores_por_tag(results_df, tags_list)
print("Valores encontrados:", tags_valores)

fab_fecha, cad_fecha = extraer_fechas(results_df)
print("Fecha de Fabricacion (FAB):", fab_fecha)
print("Fecha de Caducidad(CAD):", cad_fecha)

Valores encontrados: {'REF': '(240)07254733190', 'LOT': '(10)81259101'}
Fecha de Fabricacion (FAB): 2024-08-02
Fecha de Caducidad(CAD): 2027-05-31
